In [ ]:
import tensorflow as tf
from sklearn.preprocessing import OrdinalEncoder
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Bidirectional, LSTM

In [ ]:
data = pd.read_csv("donnees_prix.csv")

In [ ]:
def get_mois(date):
  return int(date.split("/")[0])
def get_annee(date):
  return int(date.split("/")[1])
def get_produit(text):
  return text.upper()

In [ ]:
data["produits"] = data["produits"].apply(get_produit)
data["mois"] = data["date"].apply(get_mois)
data["annee"] = data["date"].apply(get_annee)

### Transform categorical columns and create sequences

In [ ]:
encoder = OrdinalEncoder()
encoder.fit(data[["produits","regions"]])

data[["produits","regions"]] = encoder.transform(data[["produits","regions"]])

data = data[["produits","regions","mois","annee","prix/KG"]]

In [ ]:
grouped = data.groupby(["produits", "regions"])

In [ ]:
def create_sequences(grouped):
  X,y = [],[]
  for group_name, grouped_df in grouped:
    grouped_array = np.array(grouped_df, dtype=np.float32)
    for i in range(len(grouped_array) - 1):
      seq_x = grouped_array[i, :]
      seq_y = grouped_array[i+1, -1]
      X.append(seq_x)
      y.append(seq_y)
  return np.array(X, dtype=np.float32),np.array(y, dtype=np.float32)

X, y = create_sequences(grouped)
X = np.expand_dims(X, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)

### Create and train model

In [ ]:
model = tf.keras.Sequential()
model.add(LSTM(96, input_shape=(X.shape[1], X.shape[2]), activation='relu'))
model.add(Dense(1, activation="linear"))

model.compile(
    optimizer="adam",
    loss="mae",
    metrics = ['mse', tf.keras.metrics.R2Score()]
)

In [ ]:
model.fit(X_train, y_train, validation_split=.2, epochs=40, verbose=1)

Epoch 1/40
281/281 [==============================] - 8s 12ms/step - loss: 78.4862 - mse: 21770.5801 - r2_score: 0.4849 - val_loss: 39.0410 - val_mse: 4691.6367 - val_r2_score: 0.8954
Epoch 2/40
281/281 [==============================] - 2s 7ms/step - loss: 35.6916 - mse: 4014.5466 - r2_score: 0.9050 - val_loss: 36.0926 - val_mse: 3885.2920 - val_r2_score: 0.9134
Epoch 3/40
281/281 [==============================] - 1s 4ms/step - loss: 33.6146 - mse: 3837.2688 - r2_score: 0.9092 - val_loss: 34.2897 - val_mse: 3998.4268 - val_r2_score: 0.9109
Epoch 4/40
281/281 [==============================] - 1s 4ms/step - loss: 33.5078 - mse: 3782.8623 - r2_score: 0.9105 - val_loss: 34.7747 - val_mse: 3648.8940 - val_r2_score: 0.9186
Epoch 5/40
281/281 [==============================] - 2s 7ms/step - loss: 32.6694 - mse: 3700.4487 - r2_score: 0.9125 - val_loss: 32.8217 - val_mse: 3749.0054 - val_r2_score: 0.9164
Epoch 6/40
281/281 [==============================] - 2s 7ms/step - loss: 32.2088 - mse:

In [ ]:
model.evaluate(X_test, y_test)

88/88 [==============================] - 0s 2ms/step - loss: 29.3161 - mse: 4102.7432 - r2_score: 0.8994


[29.316146850585938, 4102.7431640625, 0.8994147181510925]

In [ ]:
model.save("price_prediction.h5")
joblib.dump(encoder, "encoder.pkl")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['encoder.pkl']

In [ ]:
def predict(annee, mois, produit, region, prix):
  series = pd.DataFrame({
      "produits": [produit],
      "regions": [region],
      "mois": [mois],
      "annee": [annee],
      "prix/KG": [prix]
  })
  series[["produits", "regions"]] = encoder.transform(series[["produits","regions"]])
  X = np.array(series.values, dtype=np.float32)
  X = np.expand_dims(X, axis=1)
  results = model.predict(X)
  return results

var = predict(2024, 6, "PATATE", "DAKAR", 400)[0][0]

1/1 [==============================] - 0s 218ms/step


In [ ]:
type(float(var.astype(np.float64)))

float